In [1]:
from IPython.core.display import display, HTML; display(HTML("<style>.container { width:90% !important; }</style>")) 

W tej części nauczymy się w jaki sposób zautomatyzować tworzenie tabeli. Na początkowym etapie projektowania bazy danych często pojawia się potrzeba usuniecia tabeli, przerobienia jej i stworzenia od nowa. Dlatego właśnie ważne jest aby było to szybkie i automatyczne.

Zamysł jest taki, żeby informacje o tabeli (tabelach) trzymać w plikach konfiguracyjnych i na podstawie tych plików generować zapytania CREATE.

## Zapis i odczyt plików typu JSON

Pliki JSON (JavaScript Object Notation) dobrze nadają się jako pliki konfiguracyjne, w których można przechowywać np. informacje o strukturze tabel, które następnie utworzymy

In [2]:
import json

dict_to_dump = {"a": [1, 2, 3], "b": "c", "d": {"e": [4, 5], "f": "g"}}
dict_to_dump

{'a': [1, 2, 3], 'b': 'c', 'd': {'e': [4, 5], 'f': 'g'}}

In [3]:
with open("json_file.json", "w") as f:
    json.dump(dict_to_dump, f)

In [4]:
with open("json_file.json", "r") as f:
    loaded_dict = json.load(f)

In [5]:
loaded_dict

{'a': [1, 2, 3], 'b': 'c', 'd': {'e': [4, 5], 'f': 'g'}}

In [6]:
json.dump(dict_to_dump, open("json_file.json", "w"))

In [7]:
loaded_dict = json.load(open("json_file.json", "r"))

In [8]:
loaded_dict

{'a': [1, 2, 3], 'b': 'c', 'd': {'e': [4, 5], 'f': 'g'}}

Stringi w JSONach muszą znajdować się w cudzysłowiach, nie apostrofach.

## Plik konfiguracyjny tabeli

```
{"name": "cars",
 "columns": [
        {"name": "id", "type": "serial"},
        {"name": "brand", "type": "varchar(20)", "not_null": 1, "unique": 0},
        {"name": "price", "type": "numeric", "not_null": 1, "unique": 0},
        {"name": "fuel_type", "type": "varchar(12)", "not_null": 0, "unique": 0},
        {"name": "mileage", "type": "numeric", "not_null": 0, "unique": 0},
        {"name": "is_new", "type": "boolean", "not_null": 1, "unique": 0},
        {"name": "prod_date", "type": "date", "not_null": 0, "unique": 0}]
}
```

In [9]:
config = {"name": "cars",
 "columns": [
        {"name": "id", "type": "serial"},
        {"name": "brand", "type": "varchar(20)", "not_null": 1, "unique": 0},
        {"name": "price", "type": "numeric", "not_null": 1, "unique": 1},
        {"name": "fuel_type", "type": "varchar(12)", "not_null": 0, "unique": 0},
        {"name": "mileage", "type": "numeric", "not_null": 0, "unique": 0},
        {"name": "is_new", "type": "boolean", "not_null": 1, "unique": 0},
        {"name": "prod_date", "type": "date", "not_null": 0, "unique": 0}]
}

In [10]:
config

{'name': 'cars',
 'columns': [{'name': 'id', 'type': 'serial'},
  {'name': 'brand', 'type': 'varchar(20)', 'not_null': 1, 'unique': 0},
  {'name': 'price', 'type': 'numeric', 'not_null': 1, 'unique': 1},
  {'name': 'fuel_type', 'type': 'varchar(12)', 'not_null': 0, 'unique': 0},
  {'name': 'mileage', 'type': 'numeric', 'not_null': 0, 'unique': 0},
  {'name': 'is_new', 'type': 'boolean', 'not_null': 1, 'unique': 0},
  {'name': 'prod_date', 'type': 'date', 'not_null': 0, 'unique': 0}]}

## Generowanie zapytania na podstawie JSONa

``` SQL
CREATE TABLE cars (
    id          serial,             
    brand       varchar(20) NOT NULL,          
    price       numeric NOT NULL UNIQUE,              
    fuel_type   varchar(12),                 
    mileage     numeric,
    is_new      boolean NOT NULL,              
    prod_date   date                    
);
```

In [11]:
create_query = f"CREATE TABLE {config['name']} ("   # należy zwrócić uwagę na różne cudzysłowy w tej konstrukcji

In [12]:
create_query

'CREATE TABLE cars ('

In [13]:
def return_not_null(is_not_null):
    return "NOT NULL" if is_not_null else ""

    
def return_unique(is_unique):
    return "UNIQUE" if is_unique else ""

In [25]:
return_unique(1)

'UNIQUE'

In [14]:
config

{'name': 'cars',
 'columns': [{'name': 'id', 'type': 'serial'},
  {'name': 'brand', 'type': 'varchar(20)', 'not_null': 1, 'unique': 0},
  {'name': 'price', 'type': 'numeric', 'not_null': 1, 'unique': 1},
  {'name': 'fuel_type', 'type': 'varchar(12)', 'not_null': 0, 'unique': 0},
  {'name': 'mileage', 'type': 'numeric', 'not_null': 0, 'unique': 0},
  {'name': 'is_new', 'type': 'boolean', 'not_null': 1, 'unique': 0},
  {'name': 'prod_date', 'type': 'date', 'not_null': 0, 'unique': 0}]}

###### Wersja 1

In [15]:
columns_definition = ','.join([col["name"] + " " + col["type"] + " " + return_not_null(col.get("not_null", 0)) + " " + return_unique(col.get("unique", 0))
                               for col in config["columns"]])

In [16]:
columns_definition

'id serial  ,brand varchar(20) NOT NULL ,price numeric NOT NULL UNIQUE,fuel_type varchar(12)  ,mileage numeric  ,is_new boolean NOT NULL ,prod_date date  '

###### Wersja 2

In [26]:
config["columns"]

[{'name': 'id', 'type': 'serial'},
 {'name': 'brand', 'type': 'varchar(20)', 'not_null': 1, 'unique': 0},
 {'name': 'price', 'type': 'numeric', 'not_null': 1, 'unique': 1},
 {'name': 'fuel_type', 'type': 'varchar(12)', 'not_null': 0, 'unique': 0},
 {'name': 'mileage', 'type': 'numeric', 'not_null': 0, 'unique': 0},
 {'name': 'is_new', 'type': 'boolean', 'not_null': 1, 'unique': 0},
 {'name': 'prod_date', 'type': 'date', 'not_null': 0, 'unique': 0}]

In [21]:
columns_definitions = []
for col in config["columns"]:
    single_column_definition = col["name"] + " " + col["type"] + " " + return_not_null(col.get("not_null", 0)) + " " + return_unique(col.get("unique", 0))
    columns_definitions.append(single_column_definition)
    
columns_definition = ','.join(columns_definitions)

In [22]:
columns_definition

'id serial  ,brand varchar(20) NOT NULL ,price numeric NOT NULL UNIQUE,fuel_type varchar(12)  ,mileage numeric  ,is_new boolean NOT NULL ,prod_date date  '

In [23]:
create_query += columns_definition + ");"

In [27]:
create_query

'CREATE TABLE cars (id serial  ,brand varchar(20) NOT NULL ,price numeric NOT NULL UNIQUE,fuel_type varchar(12)  ,mileage numeric  ,is_new boolean NOT NULL ,prod_date date  );id serial  ,brand varchar(20) NOT NULL ,price numeric NOT NULL UNIQUE,fuel_type varchar(12)  ,mileage numeric  ,is_new boolean NOT NULL ,prod_date date  );'

W analogiczny sposób generujemy wszystkie pozostałe zapytania: INSERT, DELETE, UPDATE itd. CREATE oraz SELECT są najtrudniejsze bo najmniej schematyczne.

## Zadanie

- Zaprojektuj tabelę według poniższego opisu. Zacznij od pliku konfiguracyjnego, który zapiszesz jako json a następnie wczytasz i wykorzystasz do stworzenia zapytania
- Dobierz odpowiednie typy danych. Ich dokładny opis możesz znaleźć np. tutaj -> https://www.postgresqltutorial.com/postgresql-data-types/
- Wystarczy utworzenie tabeli, dodawanie do niej wierszy nie będzie na razie potrzebne
- Przechowuj dane do logowania (credentials) w pliku np. json i wczytuj je tak aby nie znajdowały się bezpośrednio w kodzie
- Aby upewnić się, że zapytanie jest poprawne uruchom jest w pgAdminie albo - najlepiej - z poziomu Pythona

Czas: około 25 minut

In [35]:
import json


def return_not_null(is_not_null):
    return "NOT NULL" if is_not_null else ""

    
def return_unique(is_unique):
    return "UNIQUE" if is_unique else ""
    

config = json.load(open("configs/blogposts.json", "r"))

create_query = "CREATE TABLE blogposts ("

columns_definitions = []
for col in config["columns"]:
    single_column_definition = col["name"] + " " + col["type"] + " " + return_not_null(col.get("not_null", 0)) + " " + return_unique(col.get("unique", 0))
    columns_definitions.append(single_column_definition)
    
columns_definition = ','.join(columns_definitions)

create_query += columns_definition + ");"

print(create_query)


FileNotFoundError: [Errno 2] No such file or directory: 'configs/blogposts.json'

---

Tabela ma za zadanie przechowywać informacje o wpisach na blogu. Do cech każdego wpisu powinny należeć:
- Numer identyfikacyjny, który będzie wyliczał się automatycznie
- Autor
- Data i czas powstania posta (przechowywane w jednej kolumnie)
- Lista tagów. Może ona pozostać pusta
- Treść
- Czy post był edytowany, czy nie
- Liczba komentarzy